In [1]:
from handcalcs_adapter.units import *
import math
from math import (
    sqrt, 
    sin, cos, tan, radians)

import handcalcs
from handcalcs import render
handcalcs.set_option("display_precision", 2)
handcalcs.set_option("param_columns", 1)

In [2]:
from handcalcs_adapter.section import section, Section
sec = section(Section.C200x75x20x2_8)

KeyError: 'path'

In [ ]:
%%render input

## 1. Input

## Purlin geometry
L = (8.0 * m) # Purlin span
S = (1.2 * m) # Purlin spacing
theta = (radians( 15.0 )) # Purlin roll angle from strong axis

## Sag-rod
N_sagrod = 1

## Load
## Load design
q_DL = (0.3 * kN/(m**2)) # dead load
q_LL = (0.24 * kN/(m**2)) # live load
q_WL = (0.4 * kN/(m**2)) # wind load
# Catatan:
# - Live load
#     bangunan pabrik : 0.24 kN/m2

## Material
E = (200_000 * MPa) # Young's modulus
F_y = (240 * MPa) # Yield strength


In [ ]:
%%render input

## 2. Selec purlin
section_name = (sec.name)
w_self = (sec.w) # self weight
I_x = (sec.Ix) # strong-axis moment of inertia
I_y = (sec.Iy) # weak-axis moment of inertia
Z_x = (sec.Zx) # strong-axis modulus of section
Z_y = (sec.Zy) # weak-axis modulus of section

In [ ]:
%%render report

## 3. Calculation

## 3.1 Check for deflection

## Design load for deflection
w_DL = q_DL * S
w_LL = q_LL * S

## Design load for deflection
w = w_DL + w_LL + w_self

## Load resolution for each axis
w_x = w * cos(theta)
w_y = w * sin(theta)

## Deflection allowable
Delta_allowable = L / 240

## Calculate deflection
## ---
## On X axis
L_x = L
Delta_x = (5 * w_x * L_x**4) / (384 * E * I_x)

## On Y axis
L_y = L / (N_sagrod + 1)
Delta_y = (5 * w_y   * L_y**4) / (384 * E * I_y)

## Total deflection
Delta_total = (Delta_x**2 + Delta_y**2) ** 0.5

if Delta_total <= Delta_allowable : Delta = Delta_allowable # OK
elif Delta_total > Delta_allowable : Delta = Delta_allowable # NOT.OK

## 3.1 Check for flexural strength

## Design load for flexural strength
w_DL = q_DL * S
w_LL = q_LL * S
w_WL = q_WL * S

## Design load for deflection
w1 = 1.2*(w_DL + w_self) + 1.6*w_LL + 0.5*w_WL
w2 = 1.2*(w_DL + w_self) + 1*w_LL + 1*w_WL
w = max(w1, w2)

## Load resolution for each eaxis
w_x = w * cos(theta)
w_y = w * sin(theta)

## Calculate moment ultimate
## ---
## On X axis
L_x = L
M_UX = 1/8 * w_x * (L_x ** 2)

## On Y axis
L_y = L / (N_sagrod + 1)
M_UY = 1/8 * w_y * (L_y ** 2)

## Calculate flexural strength
## ---
## On X axis
M_nX = 0.9 * F_y * Z_x

## On Y axis
M_nY = 0.9 * F_y * Z_y

if M_UX <= M_nX : M_nX = M_nX # OK
elif M_UX > M_nX : M_nX = M_nX # NOT.OK 
if M_UY <= M_nY : M_nY = M_nY # OK
elif M_UY > M_nY : M_nY = M_nY # NOT.OK 
